In [1]:
import urllib as ul
from bs4 import BeautifulSoup
import time
import re

In [2]:
url_start = 'http://www.indianmirror.com/indianbabynames/indiannames-'

In [3]:
my_user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'

In [4]:
names = {'name':[], 'desc':[],'gender':[]}

In [5]:
import string
alpha = re.findall('[a-z]',string.ascii_lowercase)

In [6]:
pages = ['','1','2','3','4','5']

In [7]:
gender = ['male','female']

In [8]:
for gen in gender:
    for l in alpha:
        for page in pages:
            try:
                req = ul.request.Request(url=url_start+gen+'-'+l+page+'.html',
                                         data = None, 
                                         headers={'User-Agent':my_user_agent})
                doc = ul.request.urlopen(req).read().decode('utf-8')
                bsObj = BeautifulSoup(doc,'lxml')
                rows = bsObj.find_all('table')[0].find_all('tr')[1:]
                for row in rows:
                    try:
                        col = row.find_all('td')
                        name = col[0].get_text()
                        desc = col[1].get_text()
                        name = re.sub("[\\n]",'',name)
                        desc = re.sub("[\\n]",'',desc)
                        names['name'].append(name)
                        names['desc'].append(desc)
                        names['gender'].append(gen)
                    except:
                        continue
                time.sleep(5)
            except:
                break
                    

In [13]:
# from sklearn.externals import joblib
# joblib.dump(names,'/home/subhasis/indian_names.pkl')
# names1 = joblib.load('/home/subhasis/indian_names.pkl')

In [9]:
df = pd.DataFrame(names)

In [10]:
from sklearn.feature_extraction.text import 

SyntaxError: invalid syntax (<ipython-input-10-8bf5cbc6a4ef>, line 1)

In [19]:
CV = CountVectorizer(token_pattern="[A-Za-z]")

In [28]:
CV.fit(df_shuffled['name'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='[A-Za-z]', tokenizer=None,
        vocabulary=None)

In [32]:
data = CV.transform(df['name'])

In [30]:
data_dense = data.todense()

In [31]:
DV = pd.get_dummies(df['gender'])

In [68]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [45]:
train_x, test_x, train_y, test_y = train_test_split(data_dense, df['gender'], test_size=0.4, random_state=12345)

In [61]:
from sklearn.ensemble import RandomForestClassifier

In [65]:
rf = RandomForestClassifier(n_estimators=500)

In [66]:
cross_val_score(rf,X=train_x, y=train_y, cv=5, n_jobs=4)

array([0.6552901 , 0.63822526, 0.65699659, 0.6331058 , 0.63481229])

In [67]:
param = {'max_depth':[3,5,7,9],'max_features':[3,7,11,15,20,24]}

In [70]:
best_rf = GridSearchCV(rf, param_grid=param, cv=5, n_jobs=4)

In [71]:
best_rf.fit(X=train_x, y=train_y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'max_depth': [3, 5, 7, 9], 'max_features': [3, 7, 11, 15, 20, 24]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [72]:
best_rf.best_params_

{'max_depth': 9, 'max_features': 7}

In [73]:
cross_val_score(best_rf.best_estimator_,X=train_x, y=train_y, cv=5, n_jobs=4)

array([0.64334471, 0.68771331, 0.64505119, 0.64334471, 0.64505119])

In [74]:
from sklearn.metrics import accuracy_score

In [75]:
pred = best_rf.best_estimator_.predict(test_x)

In [76]:
accuracy_score(pred,test_y)

0.6535312180143296

In [77]:
from sklearn.linear_model import LogisticRegression

In [78]:
lr = LogisticRegression()

In [79]:
cross_val_score(lr,X=train_x, y=train_y, cv=5, n_jobs=4)

array([0.66382253, 0.6996587 , 0.68600683, 0.68259386, 0.67064846])

In [80]:
lr.fit(train_x,train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [81]:
pred_lr = lr.predict(test_x)

In [82]:
accuracy_score(pred_lr, test_y)

0.6888433981576254

In [102]:
new_name = 'Sonia'

In [103]:
xx = CV.transform([new_name])

In [104]:
lr.predict(xx)

array(['female'], dtype=object)